In [1]:
from app.utilities.s3 import list_files, move_file
from app.utilities.database import get_job_status
from app.utilities.rabbitmq import *
from app.utilities.logging import logger

In [2]:
get_job_status('validation/in-progress/1-20250905051437-test12.csv')

'file_queued'

In [3]:
list_files('validation/queued/')

[{'Key': 'validation/queued/1-20250905051437-test12.csv',
  'LastModified': datetime.datetime(2025, 9, 5, 5, 14, 48, 73000, tzinfo=tzutc()),
  'ETag': '"70ce360f86b41e4254b7de1a70b7652d"',
  'Size': 73,
  'StorageClass': 'STANDARD'},
 {'Key': 'validation/queued/1-20250907191002-test13.csv',
  'LastModified': datetime.datetime(2025, 9, 7, 19, 10, 32, 439000, tzinfo=tzutc()),
  'ETag': '"bfec12d988cfaee58ba1a8af52c73025"',
  'Size': 88,
  'StorageClass': 'STANDARD'},
 {'Key': 'validation/queued/1-20250909223541-test14.csv',
  'LastModified': datetime.datetime(2025, 9, 10, 4, 0, 0, 109000, tzinfo=tzutc()),
  'ETag': '"650e5d0f6a3093497aeee4a88f8c4c42"',
  'Size': 145,
  'StorageClass': 'STANDARD'}]

In [4]:
queue_agent = QueueAgent(rabbitmq_vhost='mls.batch.pending_validation')

2025-09-10 17:50:06,719 - DEBUG - Connected to RabbitMQ at rabbitmq.apps.cansin.net:5672


In [5]:
queue_agent.create_queue('test-queue-123')

2025-09-10 17:50:07,447 - DEBUG - Created queue: 'test-queue-123'.


True

In [6]:
queues = queue_agent.list_all_queues()
queues

['test-queue-123']

In [7]:
for i in range(3):
    queue_agent.publish_message(queues[0], {'test': f'message{i+1}'})

2025-09-10 17:50:08,117 - DEBUG - Published message to queue 'test-queue-123'.
2025-09-10 17:50:08,327 - DEBUG - Published message to queue 'test-queue-123'.
2025-09-10 17:50:08,473 - DEBUG - Published message to queue 'test-queue-123'.


In [8]:
queue_agent.get_message_counts(queues[0])

{'ready': 6, 'unacked': 0, 'total': 6}

In [9]:
msg = queue_agent.get_message(queues[0])
msg

2025-09-10 17:50:18,233 - DEBUG - Retrieved message from queue 'test-queue-123'.


{'test': 'message1', 'delivery_tag': 1}

In [10]:
queue_agent.get_message_counts(queues[0])

{'ready': 8, 'unacked': 1, 'total': 9}

In [11]:
queue_agent.reject_message(msg)

2025-09-10 17:50:36,759 - DEBUG - Rejected message with delivery tag '1'. Requeue: True


True

In [12]:
queue_agent.get_message_counts(queues[0])

{'ready': 9, 'unacked': 0, 'total': 9}

In [13]:
msg = queue_agent.get_message(queues[0])
msg

2025-09-10 17:50:51,463 - DEBUG - Retrieved message from queue 'test-queue-123'.


{'test': 'message1', 'delivery_tag': 2}

In [14]:
queue_agent.acknowledge_message(msg)

2025-09-10 17:50:53,369 - DEBUG - Acknowledged message with delivery tag '2'.


True

In [15]:
queue_agent.get_message_counts(queues[0])

{'ready': 8, 'unacked': 0, 'total': 8}

In [16]:
queue_agent.delete_queue(queues[0])

2025-09-10 17:51:17,281 - DEBUG - Deleted queue: 'test-queue-123'.


True

In [17]:
logger.info("Prototypes ran successfully!")

2025-09-10 17:51:31,497 - INFO - Prototypes ran successfully!
